In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib
import re

In [8]:
df = pd.read_csv('accident_df.csv')

df.head()

,사고번호,사고일시,요일,시군구,도로명주소,사고내용,사망자수,중상자수,경상자수,부상신고자수,...,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,좌표,좌표계
0,2021010200100053,2021년 1월 2일 10시,토요일,서울특별시 마포구 구수동,NaN,중상사고,0,1,0,0,...,개인형이동수단(PM),남,28세,상해없음,이륜,남,49세,중상,"[194066.02579262, 449747.90669503]",EPSG:5181
1,2021011000100182,2021년 1월 10일 16시,일요일,서울특별시 서초구 반포동,NaN,경상사고,0,0,1,0,...,개인형이동수단(PM),남,24세,경상,승용,남,43세,상해없음,"[201601.91336089, 445614.34964187]",EPSG:5181
2,2021011400100148,2021년 1월 14일 10시,목요일,서울특별시 서초구 서초동,NaN,부상신고사고,0,0,0,1,...,개인형이동수단(PM),여,24세,부상신고,승용,여,39세,상해없음,"[200524.55963108, 442588.44509583]",EPSG:5181
3,2021011500100555,2021년 1월 15일 21시,금요일,서울특별시 송파구 송파동,NaN,경상사고,0,0,1,0,...,개인형이동수단(PM),남,26세,상해없음,보행자,여,30세,경상,"[209569.1932179, 445214.50896024]",EPSG:5181
4,2021011700100198,2021년 1월 17일 18시,일요일,서울특별시 강남구 논현동,NaN,경상사고,0,0,1,0,...,개인형이동수단(PM),남,34세,상해없음,보행자,여,48세,경상,"[202131.5358183, 446477.50152318]",EPSG:5181


In [19]:
df.describe()

,사고번호,도로명주소,사망자수,중상자수,경상자수,부상신고자수
count,4.450000e+02,0.0,445.000000,445.000000,445.000000,445.000000
mean,2.021069e+15,NaN,0.002247,0.238202,0.658427,0.202247
std,3.086506e+10,NaN,0.047405,0.447089,0.553614,0.413178
min,2.021010e+15,NaN,0.000000,0.000000,0.000000,0.000000
25%,2.021042e+15,NaN,0.000000,0.000000,0.000000,0.000000
50%,2.021071e+15,NaN,0.000000,0.000000,1.000000,0.000000
75%,2.021093e+15,NaN,0.000000,0.000000,1.000000,0.000000
max,2.021123e+15,NaN,1.000000,2.000000,2.000000,2.000000


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445 entries, 0 to 444
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   사고번호        445 non-null    int64  
 1   사고일시        445 non-null    object 
 2   요일          445 non-null    object 
 3   시군구         445 non-null    object 
 4   도로명주소       0 non-null      float64
 5   사고내용        445 non-null    object 
 6   사망자수        445 non-null    int64  
 7   중상자수        445 non-null    int64  
 8   경상자수        445 non-null    int64  
 9   부상신고자수      445 non-null    int64  
 10  사고유형        445 non-null    object 
 11  법규위반        445 non-null    object 
 12  노면상태        445 non-null    object 
 13  기상상태        445 non-null    object 
 14  도로형태        445 non-null    object 
 15  가해운전자 차종    445 non-null    object 
 16  가해운전자 성별    445 non-null    object 
 17  가해운전자 연령    445 non-null    object 
 18  가해운전자 상해정도  445 non-null    object 
 19  피해운전자 차종    423 non-null    o

도로명주소 확인, 사고일시 object, 연령 object, 피해운전자 차종 object

In [ ]:
import pandas as pd
import numpy as np
import re

output_file = "accident_df_preprocessed.csv"

# 연령 전처리 위한 함수
def convert_age_to_int(age_str):
    # 나이 결측값 처리
    if pd.isna(age_str) or age_str in ['미상', '불명']:
        return np.nan
    # '세'를 제거하고 숫자로 변환
    age_num = re.sub(r'[^0-9]', '', str(age_str))
    if age_num:
        return int(age_num)
    return np.nan

df_clean = df.copy()

# 사고일시에서 '시' 제거
df_clean['사고일시'] = pd.to_datetime(df_clean['사고일시'].str.replace('시', ''), format='%Y년 %m월 %d일 %H')

# 연령 칼럼들에서 age를 int로 변환(아까 만든 함수 사용)
df_clean['가해운전자 연령'] = df_clean['가해운전자 연령'].apply(convert_age_to_int).astype('Int64')
df_clean['피해운전자 연령'] = df_clean['피해운전자 연령'].apply(convert_age_to_int).astype('Int64')

# 도로명주소 제거
df_clean = df_clean.drop(columns=['도로명주소'])

# 노면상태의 '-'를 정보없음으로 변경.
df_clean['노면상태'] = df_clean['노면상태'].replace('-', '정보없음')

df_clean.to_csv(output_file, index=False)

df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445 entries, 0 to 444
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   사고번호        445 non-null    int64         
 1   사고일시        445 non-null    datetime64[ns]
 2   요일          445 non-null    object        
 3   시군구         445 non-null    object        
 4   사고내용        445 non-null    object        
 5   사망자수        445 non-null    int64         
 6   중상자수        445 non-null    int64         
 7   경상자수        445 non-null    int64         
 8   부상신고자수      445 non-null    int64         
 9   사고유형        445 non-null    object        
 10  법규위반        445 non-null    object        
 11  노면상태        445 non-null    object        
 12  기상상태        445 non-null    object        
 13  도로형태        445 non-null    object        
 14  가해운전자 차종    445 non-null    object        
 15  가해운전자 성별    445 non-null    object        
 16  가해운전자 연령    439 non-null  